In [156]:
from ReadFile import Read
import numpy as np
import astropy.units as u
from astropy.table import QTable 
from astropy.io import ascii

In [160]:
def ComponentMass(filename = str, subset = int):
    ''' 
        This function retrieves the total mass of a given subset.

        Inputs:
        -------
                filename (str): the name of the file containing information on the galaxy you wish to
                                analyze
                subset (int)  : the index referring to the type of mass wanted: 1 for Halo Mass, 2 for
                                the Halo Mass, and 3 for the Disk Mass
        
        Returns:
        --------
                mass (float) : the mass of the subset in 1e12*solMass
        '''
    #call the Read function from Homework 1
    time, total, data = Read(filename)

    #create array of all indeces pertaining to the given subset
    subset = np.where(data['type'] == subset)[0]

    #set up try-except clause to protect function from an error when dealing with the Bulge Mass of M33
    try:
        #redefine our subset term to be an array of all lines of data pertaining to the given subset
        subset = data[subset[0] : subset[-1] + 1]
    #return 0 for mass if there are no entries in the file for that subset
    except IndexError:
        return 0

    #mass = 1e10*sum(masses)/1e12 = 1e-2*sum(masses)
    #[entry[1] for entry in subset] is a for-loop that runs through all masses of objects in the subset
    mass = np.round(1e-2*np.sum(np.array([entry[1] for entry in subset])), 3)
    
    return mass


In [161]:
#compute each mass component for each galaxy
HM_MW = ComponentMass('MW_000.txt', 1)
DM_MW = ComponentMass('MW_000.txt', 2)
BM_MW = ComponentMass('MW_000.txt', 3)
HM_M31 = ComponentMass('M31_000.txt', 1)
DM_M31 = ComponentMass('M31_000.txt', 2)
BM_M31 = ComponentMass('M31_000.txt', 3)
HM_M33 = ComponentMass('M33_000.txt', 1)
DM_M33 = ComponentMass('M33_000.txt', 2)
BM_M33 = ComponentMass('M33_000.txt', 3)

In [162]:
#compute total mass of each galaxy
total_MW = np.round((HM_MW + DM_MW + BM_MW), 3)
total_M31 = np.round((HM_M31 + DM_M31 + BM_M31), 3)
total_M33 = np.round((HM_M33 + DM_M33 + BM_M33), 3)

#compute each component of local group mass
HM_LG = np.round((HM_MW + HM_M31 + HM_M33), 3)
DM_LG = np.round((DM_MW + DM_M31 + DM_M33), 3)
BM_LG = np.round((BM_MW + BM_M31 + BM_M33), 3)
total_LG = np.round((total_MW + total_M31 + total_M33), 3)

#compute stellar mass for each galaxy/group
#I know this isn't required for entry into the table but it was helpful for me to put it in
SM_MW = np.round((DM_MW + BM_MW), 3)
SM_M31 = np.round((DM_M31 + BM_M31), 3)
SM_M33 = np.round((DM_M33 + BM_M33), 3)
SM_LG = np.round((DM_LG + BM_LG), 3)

#compute baryonic fraction for each galaxy/group
f_MW = np.round(SM_MW/total_MW, 4)
f_M31 = np.round(SM_M31/total_M31, 4)
f_M33 = np.round(SM_M33/total_M33, 4)
f_LG = np.round(SM_LG/total_LG, 4)

#create astropy table using the QTable variation because we want nice units
table = QTable([['Milky Way', 'M31', 'M33', 'Local Group'], [HM_MW, HM_M31, HM_M33, HM_LG], 
        [DM_MW, DM_M31, DM_M33, DM_LG], [BM_MW, BM_M31, BM_M33, BM_LG], [SM_MW, SM_M31, SM_M33, SM_LG],
        [total_MW, total_M31, total_M33, total_LG], [f_MW, f_M31, f_M33, f_LG]], 
        names = ('Galaxy Name', 'Halo Mass', 'Disk Mass', 'Bulge Mass', 'Stellar Mass', 'Total Mass', 
        'Baryonic Fraction'), units = (None, 1e12*u.solMass, 1e12*u.solMass, 1e12*u.solMass, 
        1e12*u.solMass, 1e12*u.solMass, None))
table

Galaxy Name,Halo Mass,Disk Mass,Bulge Mass,Stellar Mass,Total Mass,Baryonic Fraction
,1e+12 solMass,1e+12 solMass,1e+12 solMass,1e+12 solMass,1e+12 solMass,
str11,float64,float64,float64,float64,float64,float64
Milky Way,1.975,0.075,0.01,0.085,2.06,0.0413
M31,1.921,0.12,0.019,0.139,2.06,0.0675
M33,0.187,0.009,0.0,0.009,0.196,0.0459
Local Group,4.083,0.204,0.029,0.233,4.316,0.054


In [163]:
#use astropy ascii package to write the latex code for our table!
ascii.write(table, Writer=ascii.Latex, latexdict=ascii.latex.latexdicts['AA'])

\begin{table}
\begin{tabular}{ccccccc}
\hline \hline
Galaxy Name & Halo Mass & Disk Mass & Bulge Mass & Stellar Mass & Total Mass & Baryonic Fraction \\
 & $\mathrm{1 \times 10^{12}\,M_{\odot}}$ & $\mathrm{1 \times 10^{12}\,M_{\odot}}$ & $\mathrm{1 \times 10^{12}\,M_{\odot}}$ & $\mathrm{1 \times 10^{12}\,M_{\odot}}$ & $\mathrm{1 \times 10^{12}\,M_{\odot}}$ &  \\
\hline
Milky Way & 1.975 & 0.075 & 0.01 & 0.085 & 2.06 & 0.0413 \\
M31 & 1.921 & 0.12 & 0.019 & 0.139 & 2.06 & 0.0675 \\
M33 & 0.187 & 0.009 & 0.0 & 0.009 & 0.196 & 0.0459 \\
Local Group & 4.083 & 0.204 & 0.029 & 0.233 & 4.316 & 0.054 \\
\hline
\end{tabular}
\end{table}
